In [1]:
!pip install emoji

In [2]:
# importing libraries required.

import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation
from keras.utils import np_utils

import matplotlib.pyplot as plt

In [3]:
# reading the csv files (train and test)

train = pd.read_csv('train_emoji.csv',header=None)
test = pd.read_csv('test_emoji.csv',header=None)

In [5]:
# printing the train and test values

train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [6]:

test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [7]:
# creating dictionaries for the emojis which we will be using
emoji_dict = { 0 : ":heart:", 1 : ":baseball:", 2 : ":smile:", 3 : ":disappointed:", 4 : ":fork_and_knife:"}


In [9]:
# displaying the emoji_dict with the emojis using emojize

for ix in emoji_dict.keys():
    print (ix,end=" ")
    print (emoji.emojize(emoji_dict[ix], use_aliases=True))
    print("\n")
 

0 ❤


1 ⚾


2 😄


3 😞


4 🍴




In [10]:
# Creating training and testing data
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

print (X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
print ("-------------------------")
print (X_train[0],Y_train[0])

(132,) (132,) (56,) (56,)
-------------------------
never talk to me again 3


In [11]:
# splitting the sentences into words from both training and test data.

for ix in range(X_train.shape[0]):
    X_train[ix] = X_train[ix].split()

for ix in range(X_test.shape[0]):
    X_test[ix] = X_test[ix].split()
    
# Converting labels into categorical form
Y_train = np_utils.to_categorical(Y_train)

C:\Users\DELL-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\DELL-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [12]:
# Now checking the above conversion by printing train and test data at 0th index
print (X_train[0],Y_train[0])

['never', 'talk', 'to', 'me', 'again'] [0. 0. 0. 1. 0.]


In [13]:
# To check maximum length of sentence in training data
np.unique(np.array([len(ix) for ix in X_train]) , return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 4,  5, 26, 35, 20, 21, 11,  5,  1,  4], dtype=int64))

In [14]:
# To check maximum length of senetence in testing data
np.unique(np.array([len(ix) for ix in X_test]) , return_counts=True)

(array([2, 3, 4, 5, 6, 7, 8]),
 array([ 3, 12, 16, 17,  3,  4,  1], dtype=int64))

In [17]:
# Glove vectors. (convert sentences into embeddings).

embeddings_index = {}

f = open('glove.6B.50d.txt', encoding='utf-8' )
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [18]:
# Checking length of a particular word
embeddings_index["i"].shape

(50,)

In [19]:
from scipy import spatial
# Checking cosine similarity of words happy and sad
spatial.distance.cosine(embeddings_index["love"], embeddings_index["hate"])

0.35413944721221924

In [20]:
# Convert sentences into vectors, (embedding layer output).

embedding_matrix_train = np.zeros((X_train.shape[0], 10, 50))  # mostly 10 words in a sentence is there
embedding_matrix_test = np.zeros((X_test.shape[0], 10, 50))

for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        embedding_matrix_train[ix][ij] = embeddings_index[X_train[ix][ij].lower()]
        
for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        embedding_matrix_test[ix][ij] = embeddings_index[X_test[ix][ij].lower()]

In [21]:
print (embedding_matrix_train.shape, embedding_matrix_test.shape)

(132, 10, 50) (56, 10, 50)


In [22]:
# TRAINING

# USING SIMPLE RNN.

model = Sequential()
model.add(SimpleRNN(64, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(SimpleRNN(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 10, 64)            7360      
_________________________________________________________________
dropout (Dropout)            (None, 10, 64)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
_________________________________________________________________
activation (Activation)      (None, 5)                 0         
Total params: 15,941
Trainable params: 15,941
Non-trainable params: 0
____________________________________________________

In [23]:
# Setting Loss and Optimiser for the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
# Training of the model and Setting hyperparameters for the model
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
5/5 [==============================] - 0s 22ms/step - loss: 1.8663 - accuracy: 0.2121
Epoch 2/50
5/5 [==============================] - 0s 8ms/step - loss: 1.8443 - accuracy: 0.2197
Epoch 3/50
5/5 [==============================] - 0s 9ms/step - loss: 1.5859 - accuracy: 0.3258
Epoch 4/50
5/5 [==============================] - 0s 9ms/step - loss: 1.5945 - accuracy: 0.3561
Epoch 5/50
5/5 [==============================] - 0s 8ms/step - loss: 1.5796 - accuracy: 0.3333
Epoch 6/50
5/5 [==============================] - 0s 8ms/step - loss: 1.4665 - accuracy: 0.4091
Epoch 7/50
5/5 [==============================] - 0s 8ms/step - loss: 1.4035 - accuracy: 0.3939
Epoch 8/50
5/5 [==============================] - 0s 8ms/step - loss: 1.4028 - accuracy: 0.4015
Epoch 9/50
5/5 [==============================] - 0s 8ms/step - loss: 1.3416 - accuracy: 0.4242
Epoch 10/50
5/5 [==============================] - 0s 8ms/step - loss: 1.1901 - accuracy: 0.5530
Epoch 11/50
5/5 [=====================

In [25]:
# Prediction of the trained model 
pred = model.predict_classes(embedding_matrix_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [32]:
# accuracy
#model.evaluate(embedding_matrix_test,Y_test)

float(sum(pred==Y_test))/embedding_matrix_test.shape[0]

0.6071428571428571

In [36]:
# predicting the sentences (outputting the labeled and predicted emojis)

for ix in range(embedding_matrix_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        #print(ix)
        print (test[0][ix],end=" ")
        print (emoji.emojize(emoji_dict[pred[ix]], use_aliases=True),end="  ")
        print (emoji.emojize(emoji_dict[Y_test[ix]], use_aliases=True))
        print("\n")

['he', 'got', 'a', 'raise'] ❤  😄


['he', 'is', 'a', 'good', 'friend'] 😄  ❤


['I', 'am', 'upset'] 😞  ❤


['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 🍴  ❤


['This', 'girl', 'is', 'messing', 'with', 'me'] 😄  😞


['are', 'you', 'serious', 'ha', 'ha'] 😞  😄


['stop', 'messing', 'around'] ⚾  😞


['you', 'brighten', 'my', 'day'] 🍴  😄


['I', 'worked', 'during', 'my', 'birthday'] 🍴  😞


['enjoy', 'your', 'break'] 😞  😄


['valentine', 'day', 'is', 'near'] 😄  ❤


['My', 'life', 'is', 'so', 'boring'] 😄  😞


['he', 'can', 'pitch', 'really', 'well'] 😞  ⚾


['I', 'like', 'your', 'jacket'] ❤  😄


['I', 'love', 'to', 'the', 'stars', 'and', 'back'] ⚾  ❤


['What', 'you', 'did', 'was', 'awesome'] 😞  😄


['I', 'want', 'to', 'joke'] ❤  😄


['go', 'away'] ⚾  😞


['yesterday', 'we', 'lost', 'again'] ⚾  😞


['family', 'is', 'all', 'I', 'have'] 😞  ❤


['You', 'totally', 'deserve', 'this', 'prize'] 🍴  😄


['I', 'did', 'not', 'have', 'breakfast'] 🍴  😞




In [37]:
# Predicting for our random sentence
x = ['i', 'do', 'think','this', 'class', 'is', 'very', 'interesting']

x_ = np.zeros((1,10,50))

for ix in range(len(x)):
    x_[0][ix] = embeddings_index[x[ix].lower()]


In [38]:
model.predict_classes(x_)

array([0], dtype=int64)

In [39]:
# using stacked LSTM.

model = Sequential()
model.add(LSTM(128, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 128)           91648     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 223,877
Trainable params: 223,877
Non-trainable params: 0
________________________________________________

In [44]:
# Setting Loss ,Optimiser for model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training model
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 150, batch_size=32,shuffle=True
                )

Epoch 1/150
5/5 [==============================] - 0s 24ms/step - loss: 0.1411 - accuracy: 0.9470
Epoch 2/150
5/5 [==============================] - 0s 25ms/step - loss: 0.0731 - accuracy: 0.9848
Epoch 3/150
5/5 [==============================] - 0s 26ms/step - loss: 0.0562 - accuracy: 0.9773
Epoch 4/150
5/5 [==============================] - 0s 22ms/step - loss: 0.0382 - accuracy: 0.9848
Epoch 5/150
5/5 [==============================] - 0s 22ms/step - loss: 0.0235 - accuracy: 1.0000
Epoch 6/150
5/5 [==============================] - 0s 21ms/step - loss: 0.0232 - accuracy: 0.9924
Epoch 7/150
5/5 [==============================] - 0s 24ms/step - loss: 0.0095 - accuracy: 1.0000
Epoch 8/150
5/5 [==============================] - 0s 23ms/step - loss: 0.0090 - accuracy: 1.0000
Epoch 9/150
5/5 [==============================] - 0s 24ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 10/150
5/5 [==============================] - 0s 23ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 11/150
5/5 [=

5/5 [==============================] - 0s 23ms/step - loss: 1.7831e-04 - accuracy: 1.0000
Epoch 82/150
5/5 [==============================] - 0s 21ms/step - loss: 1.2911e-04 - accuracy: 1.0000
Epoch 83/150
5/5 [==============================] - 0s 21ms/step - loss: 1.4320e-04 - accuracy: 1.0000
Epoch 84/150
5/5 [==============================] - 0s 20ms/step - loss: 1.0370e-04 - accuracy: 1.0000
Epoch 85/150
5/5 [==============================] - 0s 23ms/step - loss: 1.6991e-04 - accuracy: 1.0000
Epoch 86/150
5/5 [==============================] - 0s 21ms/step - loss: 2.3218e-04 - accuracy: 1.0000
Epoch 87/150
5/5 [==============================] - 0s 22ms/step - loss: 1.7935e-04 - accuracy: 1.0000
Epoch 88/150
5/5 [==============================] - 0s 21ms/step - loss: 1.2182e-04 - accuracy: 1.0000
Epoch 89/150
5/5 [==============================] - 0s 21ms/step - loss: 2.7005e-04 - accuracy: 1.0000
Epoch 90/150
5/5 [==============================] - 0s 23ms/step - loss: 2.3994e-04 - 

In [45]:
# Prediction of trained model
pred = model.predict_classes(embedding_matrix_test)


# Calculating accuracy / score  of the model
float(sum(pred==Y_test))/embedding_matrix_test.shape[0]

0.6428571428571429

In [47]:
# Printing the sentences with the predicted and the labelled emoji
for ix in range(embedding_matrix_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        #print(ix)
        print (test[0][ix],end=" ")
        print (emoji.emojize(emoji_dict[pred[ix]], use_aliases=True),end="  ")
        print (emoji.emojize(emoji_dict[Y_test[ix]], use_aliases=True))
        print("\n")

['he', 'got', 'a', 'raise'] 😞  😄


['he', 'is', 'a', 'good', 'friend'] 😄  ❤


['I', 'am', 'upset'] 😞  ❤


['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 😄  ❤


['work', 'is', 'hard'] 😄  😞


['This', 'girl', 'is', 'messing', 'with', 'me'] ❤  😞


['are', 'you', 'serious', 'ha', 'ha'] 😞  😄


['any', 'suggestions', 'for', 'dinner'] 😄  🍴


['you', 'brighten', 'my', 'day'] ❤  😄


['she', 'is', 'a', 'bully'] ❤  😞


['I', 'worked', 'during', 'my', 'birthday'] 😄  😞


['enjoy', 'your', 'break'] ⚾  😄


['valentine', 'day', 'is', 'near'] 😄  ❤


['will', 'you', 'be', 'my', 'valentine'] 😄  ❤


['I', 'will', 'go', 'dance'] ⚾  😄


['I', 'like', 'your', 'jacket'] ❤  😄


['I', 'want', 'to', 'joke'] 😞  😄


['go', 'away'] ⚾  😞


['You', 'totally', 'deserve', 'this', 'prize'] 😞  😄


['I', 'did', 'not', 'have', 'breakfast'] ❤  😞


